In [1]:
import pandas as pd
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

In [2]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
df=pd.read_csv(r'C:\mine\guitar_notes_classification\audiofiles\preprocessed_total.csv')

In [3]:
df

,feature,class
0,[-4.1302170e+02 1.4175392e+02 2.6291016e+01 ...,Dha
1,[-4.1302170e+02 1.4175392e+02 2.6291016e+01 ...,Dha
2,[-4.1302170e+02 1.4175392e+02 2.6291016e+01 ...,Dha
3,[-4.1302170e+02 1.4175392e+02 2.6291016e+01 ...,Dha
4,[-4.1302170e+02 1.4175392e+02 2.6291016e+01 ...,Dha
...,...,...
38759,[-344.25223 163.62611 13.576447 1...,sa
38760,[-344.25223 163.62611 13.576447 1...,sa
38761,[-344.25223 163.62611 13.576447 1...,sa
38762,[-344.25223 163.62611 13.576447 1...,sa


In [4]:

for j in range(len(df.feature)):
    l1=[]
    for i in (df.feature[j].strip('][').split(' ')):
        if i !='':
            l1.append(float(i.replace('\n','')))
    df.feature[j]=l1

In [5]:
X=np.array(df['feature'].tolist())
y=np.array(df['class'].tolist())

In [6]:
from sklearn.preprocessing import MinMaxScaler
msc=MinMaxScaler()
X=msc.fit_transform(X)

In [7]:
X.shape

(38764, 40)

In [8]:
y

array(['Dha', 'Dha', 'Dha', ..., 'sa', 'sa', 'sa'], dtype='<U3')

In [9]:
labelencoder=LabelEncoder()
y=labelencoder.fit_transform(y)

In [10]:
y

array([0, 0, 0, ..., 6, 6, 6], dtype=int64)

In [11]:
y.shape

(38764,)

In [12]:
y=y.reshape(y.shape[0],-1)
y.shape

(38764, 1)

In [13]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
num_labels=y.shape[1]


In [96]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [52]:
num_epochs = 100
num_batch_size = 16

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification_ann_all.hdf5',
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)

Epoch 1/100
382/384 [============================>.] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.1275
Epoch 1: val_loss improved from inf to 0.00000, saving model to saved_models\audio_classification_ann_all.hdf5


KeyboardInterrupt: 

In [16]:
num_epochs = 100
num_batch_size = 16

In [14]:

xtrainl=X_train.reshape(-1,1,X_train.shape[1])
xtestl=X_test.reshape(-1,1,X_test.shape[1])
ytrainl=y_train.reshape(-1,1,1)
ytestl=y_test.reshape(-1,1,1)

In [17]:
from tensorflow.keras.layers import LSTM
model = Sequential()
model.add(LSTM(40, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu',return_sequences=True))
model.add(Dropout(.2))
model.add(LSTM(50, activation='relu',return_sequences=True))
model.add(Dropout(.2))
model.add(LSTM(20, activation='relu'))
model.add(Dense(7))

model.compile(optimizer='adam', loss='mse')
checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification_lstm_all.hdf5',
                               verbose=1, save_best_only=True)
# fit model
model.fit(xtrainl, ytrainl, batch_size=num_batch_size, epochs=num_epochs, validation_data=(xtestl, ytestl), callbacks=[checkpointer], verbose=1)

Epoch 1/100
1939/1939 [==============================] - ETA: 0s - loss: 0.7139
Epoch 1: val_loss improved from inf to 0.08786, saving model to saved_models\audio_classification_lstm_all.hdf5
1939/1939 [==============================] - 51s 23ms/step - loss: 0.7139 - val_loss: 0.0879
Epoch 2/100
1938/1939 [============================>.] - ETA: 0s - loss: 0.1099
Epoch 2: val_loss improved from 0.08786 to 0.06054, saving model to saved_models\audio_classification_lstm_all.hdf5
1939/1939 [==============================] - 41s 21ms/step - loss: 0.1098 - val_loss: 0.0605
Epoch 3/100
1939/1939 [==============================] - ETA: 0s - loss: 0.0698
Epoch 3: val_loss improved from 0.06054 to 0.03203, saving model to saved_models\audio_classification_lstm_all.hdf5
1939/1939 [==============================] - 39s 20ms/step - loss: 0.0698 - val_loss: 0.0320
Epoch 4/100
1939/1939 [==============================] - ETA: 0s - loss: 0.0366
Epoch 4: val_loss improved from 0.03203 to 0.01863, savin

1937/1939 [============================>.] - ETA: 0s - loss: 3.0897e-04
Epoch 30: val_loss did not improve from 0.00002
1939/1939 [==============================] - 33s 17ms/step - loss: 3.0902e-04 - val_loss: 0.0019
Epoch 31/100
1937/1939 [============================>.] - ETA: 0s - loss: 1.5068e-04
Epoch 31: val_loss did not improve from 0.00002
1939/1939 [==============================] - 43s 22ms/step - loss: 1.5060e-04 - val_loss: 1.1246e-04
Epoch 32/100
1937/1939 [============================>.] - ETA: 0s - loss: 2.1095e-04
Epoch 32: val_loss did not improve from 0.00002
1939/1939 [==============================] - 40s 20ms/step - loss: 2.1086e-04 - val_loss: 6.9808e-05
Epoch 33/100
1938/1939 [============================>.] - ETA: 0s - loss: 1.9594e-04
Epoch 33: val_loss did not improve from 0.00002
1939/1939 [==============================] - 28s 14ms/step - loss: 1.9594e-04 - val_loss: 9.8222e-05
Epoch 34/100
1939/1939 [==============================] - ETA: 0s - loss: 2.0717e

KeyboardInterrupt: 

In [18]:
pred=model.predict(xtestl)

243/243 [==============================] - 2s 4ms/step


In [19]:
pred=[np.argmax(pred[i]) for i in range(len(pred))]

In [20]:
ytestl=[ytestl[i][0][0] for i in range(len(ytestl))]

In [21]:
pred

[4,
 1,
 4,
 3,
 4,
 4,
 4,
 0,
 4,
 0,
 3,
 6,
 3,
 6,
 1,
 6,
 6,
 0,
 3,
 1,
 0,
 3,
 4,
 6,
 4,
 4,
 4,
 4,
 4,
 6,
 6,
 6,
 3,
 0,
 6,
 0,
 6,
 6,
 4,
 3,
 6,
 6,
 6,
 6,
 3,
 6,
 6,
 6,
 0,
 3,
 0,
 6,
 3,
 4,
 4,
 4,
 4,
 4,
 0,
 6,
 4,
 6,
 3,
 3,
 6,
 4,
 6,
 4,
 6,
 6,
 6,
 0,
 2,
 0,
 1,
 6,
 4,
 3,
 6,
 1,
 3,
 4,
 4,
 4,
 0,
 6,
 6,
 6,
 1,
 6,
 0,
 4,
 1,
 4,
 1,
 4,
 1,
 3,
 6,
 3,
 4,
 4,
 4,
 4,
 2,
 6,
 6,
 6,
 6,
 6,
 6,
 4,
 4,
 4,
 6,
 6,
 0,
 4,
 0,
 4,
 6,
 4,
 6,
 3,
 3,
 1,
 1,
 4,
 1,
 3,
 0,
 1,
 6,
 6,
 4,
 6,
 4,
 4,
 0,
 0,
 6,
 0,
 0,
 3,
 6,
 3,
 4,
 4,
 4,
 3,
 1,
 1,
 4,
 6,
 3,
 6,
 6,
 6,
 6,
 4,
 6,
 0,
 3,
 0,
 0,
 0,
 6,
 4,
 1,
 6,
 6,
 4,
 6,
 1,
 4,
 6,
 6,
 1,
 6,
 4,
 6,
 2,
 4,
 6,
 3,
 6,
 2,
 4,
 3,
 6,
 0,
 3,
 6,
 3,
 0,
 4,
 3,
 6,
 6,
 6,
 4,
 1,
 3,
 4,
 0,
 6,
 3,
 3,
 6,
 2,
 6,
 1,
 4,
 6,
 4,
 3,
 6,
 4,
 6,
 2,
 1,
 4,
 1,
 4,
 4,
 6,
 6,
 0,
 0,
 6,
 4,
 3,
 6,
 6,
 6,
 3,
 4,
 4,
 0,
 4,
 6,
 1,
 3,
 0,
 4,
 3,
 6,
 0,
 6,
 6,


In [22]:
ytestl

[1,
 6,
 1,
 6,
 1,
 4,
 4,
 5,
 4,
 5,
 6,
 3,
 6,
 0,
 2,
 0,
 3,
 5,
 6,
 2,
 5,
 6,
 1,
 0,
 1,
 1,
 1,
 4,
 4,
 5,
 0,
 5,
 6,
 5,
 0,
 5,
 0,
 0,
 4,
 2,
 3,
 5,
 0,
 3,
 6,
 3,
 3,
 3,
 5,
 6,
 5,
 0,
 6,
 1,
 1,
 4,
 1,
 4,
 5,
 5,
 1,
 3,
 2,
 2,
 0,
 4,
 0,
 4,
 0,
 3,
 3,
 5,
 2,
 5,
 2,
 3,
 1,
 2,
 3,
 2,
 6,
 4,
 1,
 4,
 5,
 0,
 0,
 3,
 2,
 3,
 5,
 4,
 2,
 4,
 2,
 4,
 2,
 6,
 0,
 6,
 4,
 4,
 1,
 1,
 2,
 0,
 3,
 5,
 3,
 3,
 3,
 4,
 4,
 4,
 0,
 3,
 5,
 1,
 5,
 1,
 0,
 5,
 0,
 6,
 6,
 2,
 2,
 1,
 6,
 6,
 5,
 2,
 3,
 0,
 4,
 3,
 4,
 1,
 5,
 5,
 0,
 5,
 5,
 2,
 3,
 6,
 4,
 1,
 4,
 2,
 2,
 2,
 1,
 0,
 2,
 3,
 3,
 5,
 3,
 5,
 3,
 5,
 6,
 5,
 5,
 5,
 4,
 1,
 2,
 0,
 0,
 1,
 3,
 6,
 1,
 5,
 0,
 2,
 5,
 1,
 3,
 2,
 1,
 5,
 2,
 0,
 2,
 4,
 6,
 3,
 5,
 6,
 5,
 6,
 5,
 1,
 6,
 3,
 3,
 0,
 4,
 2,
 6,
 4,
 5,
 0,
 2,
 6,
 0,
 2,
 5,
 2,
 4,
 3,
 1,
 6,
 3,
 1,
 3,
 2,
 6,
 4,
 2,
 4,
 1,
 0,
 3,
 5,
 5,
 0,
 1,
 2,
 3,
 0,
 0,
 2,
 4,
 4,
 5,
 4,
 0,
 2,
 2,
 5,
 4,
 6,
 0,
 5,
 0,
 3,


In [23]:
from sklearn.metrics import accuracy_score
accuracy_score(ytestl,pred)

0.17980136721269185

In [24]:
import librosa
filename=r"C:\mine\guitar_notes_classification\sa_sample.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
# mfccs_scaled_features=(msc.fit_transform(mfccs_scaled_features)).reshape(-1,1)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,1,40)

predicted_label=model.predict(mfccs_scaled_features)

print(np.argmax(predicted_label))
# prediction_class = labelencoder.inverse_transform(predicted_label) 
# prediction_class

1/1 [==============================] - 0s 29ms/step
4
